<a href="https://colab.research.google.com/github/VodkaSin/Cuda_C_spin_sync/blob/main/CUDA_code/CUDA_spin_cyn_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/FYP/Cuda_C_spin_sync/CUDA_code")

In [ ]:
# Clone repo and test
# !git clone https://github.com/VodkaSin/Cuda_C_spin_sync.git
!ls

In [ ]:
#############################################################
# Compile `main.cu` to `file.exe`
# Commented out because we don't want to run it all the time
#############################################################
# !nvcc -w functions.cu main.cu -o file
!nvcc -w functions.cu main_sz.cu -o file

In [ ]:
#############################################################
# Load python dependencies
# If package cannot be resolved from pylance, reload the interpreter
#############################################################
import numpy as np
import matplotlib.pyplot as plt
import utils
import os.path
import scipy.stats as stats
from matplotlib import rc
!chmod 755 ./file

In [ ]:
#############################################################
# Figure settings
#############################################################
plt.rcParams.update({'font.size': 16})
# plt.rcParams["font.family"] = "Arial"

# Test run

In [ ]:
#############################################################
# Sample input
#############################################################
detunings = [[50,10]]
thetas = [[0.5,0.5]] # Value between 0 and 1 for ground and excited
# Generate a list of detuning profile with k = 10,20,30,40,50
N_tot = 50000
theta_0 = 1.0
phi_0 = 0.0
coup_a_0 = 1.0
gamma_a_0 = 0.0
chi_a_0 = 0.0
kappa_c_0 = 100.0
t_max = 0.2
t_num = 40000
det_dict = {}
handle_list = []

for i in range(len(detunings)):
    np.savetxt(f"Detuning.dat", detunings[i], delimiter="\t")
    np.savetxt(f"Sz_init.dat", thetas[i], delimiter="\t")
    num_ens = len(detunings[i])
    params = str((f'{num_ens} {N_tot} {theta_0} {phi_0} {coup_a_0} ')
                +(f'{gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num}'))
    handle = f'k{num_ens}N{N_tot}det{500}_{theta_0}_{coup_a_0}_{kappa_c_0}_{i+1}'
    det_dict[handle] = detunings[i]
    handle_list.append(handle)
    print(f"************************* Run {i+1} *************************")
    print(f"Handle: {handle} ")
    if os.path.isfile(f"Result_Sz_{handle}.dat") == False:
        !./file {params} {handle}
    else:
        print("File exits, skipping simulation")
    # Uncomment below if you wish to overwrite files
    # !./file {params} {handle}
    print("\n")

In [ ]:
#############################################################
# Read outputs
#############################################################
n_tests = len(handle_list) # Number of tests
results = [utils.read_results(handle) for handle in handle_list]
time_list = [results[i][0] for i in range(n_tests)]
sz_list = [results[i][1] for i in range(n_tests)] # Population inversion
coherence_list = [results[i][2] for i in range(n_tests)] # Coherence with class 0
photon_list = [results[i][3] for i in range(n_tests)] # Intracavity photon number

In [ ]:
sz_list[0]

# Test 5

In [ ]:
#############################################################
# Test 5: Find critical detuning
# Criteria: - clas 0.99 + class 0.99
#############################################################
N_tot = 300000
shifts = np.linspace(20,800,20).astype(int) # General run
# shifts = np.linspace(380,410,10).astype(int) # Special run 1
# shifts = np.linspace(380,410,10).astype(int) # Special run 2
# shifts = np.linspace(340,430,10).astype(int) # Special run 3
num_ens = 3
theta_0 = 1
np.savetxt(f"Sz_init.dat", [theta_0 for i in range(num_ens)], delimiter="\t")
phi_0 = 0.0
coup_a_0 = 1.0
gamma_a_0 = 0.0
chi_a_0 = 0.0
kappa_c_0 = 100.0
t_max = 0.2
t_num = 60000
det_dict = {}
handle_list = []
delta_crit = []
delta_min = 0
delta_max = 650
delta_cur = delta_max # Temporary variable
cor_cri = 0.99
tol = 1e-3 # Tolerance 0.001
# delta_crit = [] # Storing critical detunings satisfying crit

for shift in shifts:
    # Run initial test to see if the current range is ok
    crit1 = 0
    found1 = False
    run = 0
    max_run = 10
    delta_min = 0
    delta_max = delta_cur # Larger shifts have smaller critical detuning
    
    # Initial check: crit1 fails at delta_max
    while(True):
        detuning_max = [shift, -delta_max + shift, delta_max + shift]
        print(detuning_max)
        np.savetxt("Detuning.dat", detuning_max, delimiter="\t")
        params = f'{num_ens} {N_tot} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num}'
        handle_max = f'k{num_ens}N{N_tot}det{delta_max}D{shift}_{theta_0}_{coup_a_0}_{kappa_c_0}_test5'
        print(handle_max)
        # Check if file already exists
        if os.path.isfile(f"Result_Sz_{handle_max}.dat") == False:
            !./file {params} {handle_max}
        else:
            print("File exits, skipping simulation")
        # !./file {params} {handle_max}
        result = utils.read_results(handle_max)
        cor1 = np.corrcoef(result[1][:,0],result[1][:,1])[0][1]
        cor2 = np.corrcoef(result[1][:,0],result[1][:,2])[0][1]
        
        if np.abs(cor1-cor_cri) < tol and np.abs(cor2-cor_cri) < tol: # delta_max is the critical detuning for crit1
            delta_crit.append(delta_max)
            handle_list.append(handle_max)
            det_dict[handle_max] = detuning_max
            found1 = True
            delta_cur = delta_max
            print(f"delta_crit1 has been found at boundary {delta_max}, {cor}")
            break
        elif cor2 < cor_cri:
            break
        elif cor2 > cor_cri: # delta_max is smaller than critical detuning for crit1
            delta_max += 100
            print(f"delta_max smaller than critical, increase by 100 to {delta_max}")
        # break (auto) delta_max is larger than critical detuning for crit 1
    
    # Start searching by binary
    while((found1== False) and run<max_run and delta_max-delta_min>5):
        run += 1
        delta_mid = int((delta_max + delta_min)/2)
        detuning_mid = [shift, -delta_mid + shift, delta_mid + shift]
        np.savetxt("Detuning.dat", detuning_mid, delimiter="\t")   
        handle_mid = f'k{num_ens}N{N_tot}det{delta_mid}D{shift}_{theta_0}_{coup_a_0}_{kappa_c_0}_test3' # Should be test 5 instead
        print(f"************************* {N_tot} Run {run} Detuning {delta_mid} *************************")
        print(f"Handle: {handle_mid}")
        if os.path.isfile(f"Result_Sz_{handle_mid}.dat") == False:
            !./file {params} {handle_mid}
        else:
            print("File exits, skipping simulation")
        # !./file {params} {handle_mid}
        result = utils.read_results(handle_mid)
        cor1 = np.corrcoef(result[1][:,0],result[1][:,1])[0][1]
        cor2 = np.corrcoef(result[1][:,0],result[1][:,2])[0][1]
        if np.abs(cor1-cor_cri) < tol and np.abs(cor2-cor_cri) < tol:
            delta_crit.append(delta_mid)
            handle_list.append(handle_mid)
            det_dict[handle_mid] = detuning_mid
            found1 = True
            delta_cur = delta_mid
            print(f"delta_crit1 has been found at run {run}: {delta_mid}, {cor1,cor2}")
            break
        elif cor2 < cor_cri:
            delta_max = delta_mid
            print(f"Detuning {delta_mid} too large, {cor1,cor2}, next round")
        elif cor2 > cor_cri:
            delta_min = delta_mid
            print(f"Detuning {delta_mid} too small, {cor1,cor2}, next round")
    
    if found1 == False:
        delta_mid = int((delta_min + delta_max)/2)
        delta_crit.append(delta_mid)
        handle_list.append(handle_mid)
        det_dict[handle_mid] = detuning_mid
        found1 = True
        delta_cur = delta_mid
        print(f"Unable to resolve within max_iter, settle at {delta_mid} with cor ~{cor1,cor2}")

In [ ]:

#from sklearn.metrics import r2_score
from scipy.optimize import curve_fit

X = shifts
Y = delta_crit

def exp_func(X, a, b, c):
    return a * np.exp(-b*X) + c

def power_func(X, a, b) :
    return a * (X) ** (-b)

def sigmoid_func(X, a, b, c, d):
    return a / (c + np.exp(b * X)) + d

def log_func(X, a, b):
    return a* np.log(X) + b

funcs = [exp_func, power_func, sigmoid_func, log_func]
coef_list = []
# for i in range(len(funcs)):
#   print(f"Run{i}")
#   coef, _ = curve_fit(funcs[i], X, Y)
#   coef_list.append(coef)
#   print(f"Optimal param: {coef}")

popt_exp, _ = curve_fit(exp_func, X, Y, (500, 1e-3, -200))
print(popt_exp)

def predict(func, X, coe):
  return func(X, *coe)



In [ ]:
plt.scatter(shifts, delta_crit, label="Data")
plt.plot(shifts, predict(funcs[0], shifts, popt_exp))
print(shifts, delta_crit)

In [ ]:
save_fig = True
plt.figure(figsize=(8,6))
plt.scatter(shifts, delta_crit, label="Data")
# plt.plot(N_tot_range, predict_Y_log(N_tot_range))
# plt.plot(N_tot_range, predict_Y_quad(N_tot_range))
plt.plot(shifts, predict(funcs[0], shifts, popt_exp), label="Best fit", color="#ff7f0e")
plt.ylabel(r"$\delta^*$(kHz)")
plt.xlabel("$\Delta$ (kHz)")
plt.legend()
if save_fig:
  os.chdir("/content/gdrive/MyDrive/FYP/Cuda_C_spin_sync/CUDA_code/plots")
  plt.savefig(f"test5_fitcurve.pdf", dpi=300, pad_inches=1)
  os.chdir("/content/gdrive/MyDrive/FYP/Cuda_C_spin_sync/CUDA_code")
# delta_crit
# plt.scatter(N_tot_range, delta_crit)
# plt.plot(N_tot_range, predict_Y(N_tot_range))

## Special tests

In [ ]:
delta_crit = [239,254,276,305,328,
                360,402,454,519,619,
                179,156,136,119,111,
                104,104,91,91,91, 
                325, 325, 325, 325, 243, 
                243, 243, 243, 243, 182,
                507, 531, 552, 326, 326,
                326, 244, 183, 183, 183]
shifts = np.concatenate((np.linspace(20,800,20).astype(int), 
                         np.linspace(380,410,10).astype(int),
                         np.linspace(340,430,10).astype(int)))
sort_ind = np.argsort(shifts)
shifts_sort = shifts[sort_ind]
delta_sort = np.asarray(delta_crit)[sort_ind]
print(shifts_sort.reshape(8,5))
print(delta_sort.reshape(8,5))

In [ ]:
# Closer look at 300-400
inv_ind = [7,9,17,20,29,34] # Index of cases to investigate
N_tot = 300000
num_ens = 3
theta_0 = 1
np.savetxt(f"Sz_init.dat", [theta_0 for i in range(3)], delimiter="\t")
phi_0 = 0.0
coup_a_0 = 1.0
gamma_a_0 = 0.0
chi_a_0 = 0.0
kappa_c_0 = 100.0
t_max = 0.2
t_num = 60000
# det_dict = {}
# handle_list = []
cor_cri = 0.99
tol = 1e-3 # Tolerance 0.001
# delta_crit = [] # Storing critical detunings satisfying crit

det_dict = {}
handle_list = []

for i in inv_ind:
    shift = shifts_sort[i]
    delta = delta_sort[i]
    detuning = [shift, -delta + shift, delta + shift]
    np.savetxt("Detuning.dat", detuning, delimiter="\t") 
    print(np.loadtxt( 'Detuning.dat' ))
    params = f'{num_ens} {N_tot} {theta_0} {phi_0} {coup_a_0} {gamma_a_0} {chi_a_0} {kappa_c_0} {t_max} {t_num}'  
    handle = f'k{num_ens}N{N_tot}det{delta}D{shift}_{theta_0}_{coup_a_0}_{kappa_c_0}_test5' # Should be test 5 instead
    handle_list.append(handle)
    det_dict[handle] = detuning
    print(f"************************* D{shift} Detuning {delta} *************************")
    print(f"Handle: {handle}")
    if os.path.isfile(f"Result_Sz_{handle}.dat") == False:
        !./file {params} {handle}
    else:
        print("File exits, skipping simulation")
    # !./file {params} {handle}

In [ ]:
#############################################################
# Read outputs
#############################################################
n_tests = len(handle_list) # Number of tests
results = [utils.read_results(handle) for handle in handle_list]
time_list = [results[i][0] for i in range(n_tests)]
sz_list = [results[i][1] for i in range(n_tests)] # Population inversion
coherence_list = [results[i][2] for i in range(n_tests)] # Coherence with class 0
photon_list = [results[i][3] for i in range(n_tests)] # Intracavity photon number

In [ ]:
plot_save = True
fig, ax = plt.subplots(2,3,sharex=True, sharey=True, figsize=(15,8), gridspec_kw = {'wspace':0.12, 'hspace':0.08})

ax[0][0].set_ylabel(r"$\langle \sigma_z\rangle$")
ax[1][0].set_ylabel(r"$\langle \sigma_z\rangle$")
ax[1][0].set_xlabel(r"Time ($\mu$s)")
ax[1][1].set_xlabel(r"Time ($\mu$s)")
ax[1][2].set_xlabel(r"Time ($\mu$s)")
# ax[0][1].ticklabel_format(axis='y', style='sci', scilimits=(4,4))
# ax[1][1].ticklabel_format(axis='y', style='sci', scilimits=(4,4))


t_ind = utils.cut_time(time_list[0], 0.08)
index = 0
for i in range(2):
    for j in range(3):
        shift = shifts_sort[inv_ind[index]]
        delt = delta_crit[inv_ind[index]]
        labels = [f"$\delta_0=${shift}", f"$\delta_-=${shift-delt}",f"$\delta_+=${shift+delt}"]  
        ax[i][j].text(0,-0.8,f"$\Delta=${shift}\n$\delta=${delt}")
        for k in range (3):
            ax[i][j].plot(time_list[index][:t_ind], sz_list[index][:,k][:t_ind], label=labels[k])
        ax[i][j].legend()   
        index += 1
ax[0][1].set_title("Population inversion")
if plot_save:
    fig.savefig(f"test5_trend.pdf", dpi=300, pad_inches=0.2)
# i = 2
# shift = shifts[inv_ind[i]]
# delt = delta_crit[inv_ind[i]]
# plt.plot(time_list[i], sz_list[i][:,0], label=f"{shift}")
# plt.plot(time_list[i], sz_list[i][:,1], label=f"{shift-delt}")
# plt.plot(time_list[i], sz_list[i][:,2], label=f"{shift+delt}")
# plt.legend()
# plt.title(f"Shift {shift} critical detuning {delt}")